In [1]:
import requests
import pandas as pd
import math, time

In [2]:
API_KEY='qnzxhE72jAUvlnI97EAxZcYI3LLPU8qL'
BASE_URL = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

In [218]:
query_dict = {
    'Amazon':'Amazon.com Inc',
    'Apple':'Apple Inc',
    'Microsoft':'Microsoft Corp',
    'Google':'Google Inc',
    'Facebook':'Facebook Inc',
    'Tesla':'Tesla Motors Inc',
    'NVIDIA':'NVIDIA Corporation',
    'Broadcom':'Broadcom Corporation',
    'Cisco':'Cisco Systems Inc',
    'Comcast':'Comcast Corporation',
    'Intel':'Intel Corporation',
    'Adobe':'Adobe Systems Inc'}

In [194]:
def getArticleComments(article_url):
    COMMENTS_URL = 'https://api.nytimes.com/svc/community/v3/user-content/url.json'
    comm_params = {'api-key':'VOyJLAyvGqAyAHJot0I05y868muPksiG',
                'url':article_url,
                'offset':0,
                'sort':'reader'}
    
    response = requests.get(COMMENTS_URL,comm_params)
    data = response.json()

    comments = []
    
    if len(data['results']) != 0:
        for i in data['results']['comments']:
            comments.append([i['commentBody']])
    
        time.sleep(6)
        
    return comments

In [217]:
def getArticlesData(query):
    ''' Gets json data of every article from our request search parameters and extracts information 
    from each article to get full article text and store information into a dataframe

    Input: search query 
        - Company we are analyzing
    Output: Returns dataframe with each article's publishing date, url,headline, number of comments, and comments
    Criteria: The query must be in the headline and the comments are up to the 25 most recommended comments under article
    '''
    # request parameters
    page=0
    params={'api-key':API_KEY,
        'q':query,
        'begin_date':'20190101',
        'end_date':'20211231',
        'fq':f'document_type:("article") AND news_desk:("Business") AND organizations:({query_dict[query]})',
        'page':page,
        'fl':('web_url','pub_date','headline','meta'), 
        'sort':'oldest'}

    # request article data
    response = requests.get(BASE_URL,params)
    data = response.json()

    # number of articles returned
    hits = data['response']['meta']['hits']
    print(f'Hits: {hits}')
    
    # pages of articles to iterate through
    total_pages = math.ceil(hits/10)

    # initialize empty list of data
    article_data = []

    for page in range(total_pages): 
        try:
            # updates page in articles parameters for pagination
            params['page'] = page

            # gets current page's articles raw data
            response = requests.get(BASE_URL,params=params)
            data = response.json()

            # print data collection progress
            print(f'\n pg {page}/{total_pages - 1}')

            # iterate through each article's raw data on current page
            for i in data['response']['docs']:
                pub_date = i['pub_date']
                article_url = i['web_url']
                headline = i['headline']['main']
                
                # only get articles with company in headline
                if query.lower() not in headline.lower():
                    continue
                    
                # get comments of article, if any
                comments = getArticleComments(article_url)

                # filter out articles with disabled or no comments
                if len(comments) != 0:
                    article_data.append([pub_date,article_url,headline,len(comments),comments]) # add new data row

            time.sleep(6)
        except:
            print("There's an issue here")
    
    # dataframe to store article data for a company
    df = pd.DataFrame(article_data, columns=['Date','Article URL','Headline','Number of Comments','Comments'])
    df['Date'] = pd.to_datetime(df['Date']).dt.date # date formatting
    
    # save article data to its own csv file in respective company's folder
    df.to_csv(f'./ARTICLES/{query}/{query}_articles.csv',index=False,mode='w+')
    
    return df

## Data Collection

We are applying the **`getArticlesData()`** method to each company and will save the collected data to a respective csv file for later use.

Out of the articles returned from the API request, only the articles that have comments and the company's name in the headline will be saved in the dataframe.

In [219]:
getArticlesData('Adobe')

Hits: 0


,Date,Article URL,Headline,Number of Comments,Comments


In [193]:
getArticlesData('Amazon')

Hits: 518
res SUCCESS
pg 0/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 1/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 2/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 3/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 4/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 5/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 6/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 7/51
comments – SUCCESS
comments – SUCCESS
res SUCCESS
pg 8/51
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – SUCCESS
comments – S

,Date,Article URL,Headline,Number of Comments,Comments
0,2019-01-18,https://www.nytimes.com/2019/01/18/business/be...,Why Jeff Bezos’ Divorce Should Worry Amazon In...,25,[[The way we (and the press) talk about how as...
1,2019-02-14,https://www.nytimes.com/2019/02/14/business/am...,"For Once, Amazon Loses a Popularity Contest",25,[[I am against everything Amazon stands for: ...
2,2019-02-16,https://www.nytimes.com/2019/02/16/business/am...,"In Amazon Fight, Progressives Showed What They...",25,"[[Amazon uses our roads, our airports, our tra..."
3,2019-03-04,https://www.nytimes.com/2019/03/03/technology/...,Amazon’s Hard Bargain Extends Far Beyond New York,25,[[Amazon's perpetual and ongoing tax avoidance...
4,2019-05-14,https://www.nytimes.com/2019/05/14/technology/...,"Attention, Amazon Shoppers: Google Wants Some ...",25,[[I no longer use Amazon for shopping because ...
...,...,...,...,...,...
60,2021-09-28,https://www.nytimes.com/2021/09/28/technology/...,"For Flagging Amazon Games Unit, New World ‘Has...",6,"[[The word ""fun"" never appears in this article..."
61,2021-10-21,https://www.nytimes.com/2021/10/21/technology/...,Amazon Workers on Staten Island Aim for Union ...,25,[[Workers need to unite if they ever want to a...
62,2021-10-24,https://www.nytimes.com/2021/10/24/technology/...,Inside Amazon’s Worst Human Resources Problem,25,[[The contrast between Jeff Bezos' rockets tak...
63,2021-10-25,https://www.nytimes.com/2021/10/25/business/am...,Why Amazon Is in Business With Judge Judy,13,[[Am I the only one who finds Amazon’s collabo...


In [196]:
getArticlesData('Apple')

Hits: 398
pg 0/39
no comments
comments – 25
no comments
no comments
comments – 0
no comments
comments – 0
no comments
comments – 25
no comments
pg 1/39
no comments
no comments
no comments
no comments
no comments
no comments
comments – 20
comments – 0
comments – 25
comments – 25
pg 2/39
no comments
no comments
comments – 18
comments – 0
no comments
no comments
comments – 0
no comments
no comments
comments – 0
pg 3/39
comments – 0
no comments
no comments
comments – 0
no comments
no comments
comments – 0
no comments
comments – 0
no comments
pg 4/39
comments – 0
comments – 0
comments – 25
comments – 25
comments – 0
comments – 0
no comments
comments – 0
comments – 0
no comments
pg 5/39
no comments
comments – 0
comments – 25
comments – 0
no comments
no comments
comments – 19
comments – 0
no comments
comments – 25
pg 6/39
no comments
comments – 0
no comments
no comments
comments – 0
no comments
no comments
no comments
comments – 0
no comments
pg 7/39
no comments
no comments
no comments
no com

,Date,Article URL,Headline,Number of Comments,Comments
0,2019-01-02,https://www.nytimes.com/2019/01/02/technology/...,Apple Cuts Revenue Forecast Because of Slow iP...,25,"[[Yes, fewer sales in China are playing a role..."
1,2019-01-05,https://www.nytimes.com/2019/01/05/technology/...,Apple’s Biggest Problem? My Mom,25,[[As a longtime tech worker (and ex-Apple empl...
2,2019-01-29,https://www.nytimes.com/2019/01/28/technology/...,"In a Stumble for Apple, a FaceTime Bug Lets iP...",20,[[I've said it before and I'll say it again: M...
3,2019-01-30,https://www.nytimes.com/2019/01/29/technology/...,Apple Was Slow to Act on FaceTime Bug That All...,25,[[I think the teenager who found this flaw des...
4,2019-01-30,https://www.nytimes.com/2019/01/30/technology/...,How to Save on Your Next Apple Purchase,25,"[[""Of course, you can save $750 on the next $7..."
5,2019-02-01,https://www.nytimes.com/2019/01/31/technology/...,Apple Shows Facebook Who Has the Power in an A...,18,[[I hope that Apple will further restrict apps...
6,2019-03-22,https://www.nytimes.com/2019/03/22/multimedia/...,Why Netflix Won’t Be Part of Apple TV,25,[[There's already so much streaming content av...
7,2019-03-25,https://www.nytimes.com/2019/03/25/technology/...,Apple Unveils Video Service With Celebrity Par...,25,"[[Or...go to the library, take a hike, visit a..."
8,2019-04-16,https://www.nytimes.com/2019/04/16/technology/...,Apple and Qualcomm Settle All Disputes Worldwide,25,[[The missing piece of the story is why Intel ...
9,2019-04-27,https://www.nytimes.com/2019/04/27/technology/...,Apple Cracks Down on Apps That Fight iPhone Ad...,19,[[Apple needs to rethink the way they handle a...


In [200]:
getArticlesData('Facebook')

Hits: 751

 pg 0/75
0 articles found

 pg 1/75
1 articles found

 pg 2/75
0 articles found

 pg 3/75
1 articles found

 pg 4/75
0 articles found

 pg 5/75
1 articles found

 pg 6/75
1 articles found

 pg 7/75
0 articles found

 pg 8/75
0 articles found

 pg 9/75
0 articles found

 pg 10/75
0 articles found

 pg 11/75
0 articles found

 pg 12/75
0 articles found

 pg 13/75
0 articles found

 pg 14/75
0 articles found

 pg 15/75
0 articles found

 pg 16/75
0 articles found

 pg 17/75
0 articles found

 pg 18/75
1 articles found

 pg 19/75
0 articles found

 pg 20/75
0 articles found

 pg 21/75
0 articles found

 pg 22/75
0 articles found

 pg 23/75
0 articles found

 pg 24/75
0 articles found

 pg 25/75
0 articles found

 pg 26/75
1 articles found

 pg 27/75
0 articles found

 pg 28/75
0 articles found

 pg 29/75
0 articles found

 pg 30/75
0 articles found

 pg 31/75
0 articles found

 pg 32/75
0 articles found

 pg 33/75
0 articles found

 pg 34/75
0 articles found

 pg 35/75
0 article

,Date,Article URL,Headline,Number of Comments,Comments
0,2019-01-23,https://www.nytimes.com/2019/01/23/technology/...,He Reported on Facebook. Now He Approaches It ...,25,[[What would be really interesting is for the ...
1,2019-01-25,https://www.nytimes.com/2019/01/25/technology/...,"Zuckerberg Plans to Integrate WhatsApp, Instag...",25,[[If the DOJ Anti-Trust division still had a l...
2,2019-01-31,https://www.nytimes.com/2019/01/30/technology/...,Maybe Only Tim Cook Can Fix Facebook’s Privacy...,25,[[More power to Apple! The government of our ...
3,2019-02-01,https://www.nytimes.com/2019/01/31/technology/...,Apple Shows Facebook Who Has the Power in an A...,18,[[I hope that Apple will further restrict apps...
4,2019-02-07,https://www.nytimes.com/2019/02/07/technology/...,How Facebook’s Tiny China Sales Floor Helps Ge...,20,"[[If I understand this correctly, Chinese comp..."
...,...,...,...,...,...
97,2021-10-25,https://www.nytimes.com/2021/10/24/business/me...,Inside the Big Facebook Leak,25,[[Nothing would make me happier than Facebook ...
98,2021-10-25,https://www.nytimes.com/2021/10/25/technology/...,Facebook Wrestles With the Features It Used to...,25,[[How is Facebook’s behavior different than co...
99,2021-10-26,https://www.nytimes.com/2021/10/26/technology/...,Facebook Faces a Public Relations Crisis. What...,25,[[It is long past time to call a halt to these...
100,2021-10-28,https://www.nytimes.com/2021/10/28/technology/...,Facebook Renames Itself Meta,25,"[[Is it just me, or is Zuckerberg just one whi..."


In [204]:
getArticlesData('Google')

Hits: 467

 pg 0/46

 pg 1/46

 pg 2/46

 pg 3/46

 pg 4/46

 pg 5/46

 pg 6/46

 pg 7/46

 pg 8/46

 pg 9/46

 pg 10/46

 pg 11/46

 pg 12/46

 pg 13/46

 pg 14/46

 pg 15/46

 pg 16/46

 pg 17/46

 pg 18/46

 pg 19/46

 pg 20/46

 pg 21/46

 pg 22/46

 pg 23/46

 pg 24/46

 pg 25/46

 pg 26/46

 pg 27/46

 pg 28/46

 pg 29/46

 pg 30/46

 pg 31/46

 pg 32/46

 pg 33/46

 pg 34/46

 pg 35/46

 pg 36/46

 pg 37/46

 pg 38/46

 pg 39/46

 pg 40/46

 pg 41/46

 pg 42/46

 pg 43/46

 pg 44/46

 pg 45/46

 pg 46/46


,Date,Article URL,Headline,Number of Comments,Comments
0,2019-03-04,https://www.nytimes.com/2019/03/04/technology/...,Google Finds It’s Underpaying Many Men as It A...,25,[[“When Google conducted a study recently to d...
1,2019-03-20,https://www.nytimes.com/2019/03/20/business/go...,Google Fined $1.7 Billion by E.U. for Unfair A...,25,[[Excellent news !\n\nThe Europeans lead the w...
2,2019-05-14,https://www.nytimes.com/2019/05/14/technology/...,"Attention, Amazon Shoppers: Google Wants Some ...",25,[[I no longer use Amazon for shopping because ...
3,2019-05-20,https://www.nytimes.com/2019/05/20/technology/...,"U.S. Tech Suppliers, Including Google, Restric...",25,[[Google is doing exactly what americans have ...
4,2019-05-20,https://www.nytimes.com/2019/05/20/business/hu...,"As Huawei Loses Google, the U.S.-China Tech Co...",25,[[This article finally summarizes the issue ve...
5,2019-05-28,https://www.nytimes.com/2019/05/28/technology/...,Google’s Shadow Work Force: Temps Who Outnumbe...,25,[[This is not news. Every big corporation in ...
6,2019-06-02,https://www.nytimes.com/2019/06/02/business/go...,Google and Amazon Are at the Center of a Storm...,25,"[[I found this hilarious: ""President Trump an..."
7,2019-07-15,https://www.nytimes.com/2019/07/15/technology/...,A Feisty Google Adversary Tests How Much Peopl...,25,[[I'm a medical/public health researcher who t...
8,2019-07-17,https://www.nytimes.com/2019/07/17/technology/...,Google Glass May Have an Afterlife as a Device...,16,[[If you would like to learn more about the Au...
9,2019-08-27,https://www.nytimes.com/2019/08/27/technology/...,Former Star Google and Uber Engineer Charged W...,25,[[Could sometime please enlighten me as to why...


In [140]:
getArticlesData('Intel')

Hits: 31
pg 0/3
pg 1/3
pg 2/3
There's an issue here
pg 3/3


,Date,Article URL,Headline,Number of Comments,Comments
0,2020-03-01,https://www.nytimes.com/2020/03/01/technology/...,Intel’s Culture Needed Fixing. Its C.E.O. Is S...,16,[[How interesting that this may be the first c...
1,2020-08-27,https://www.nytimes.com/2020/08/27/technology/...,"Intel Slips, and a High-Profile Supercomputer ...",25,[[I fear that what we're seeing here in Intel'...
2,2020-12-01,https://www.nytimes.com/2020/12/01/technology/...,Amazon and Apple Are Powering a Shift Away Fro...,25,"[[""Homegrown"" implies that these companies m..."


In [202]:
getArticlesData('Microsoft')

Hits: 133

 pg 0/13
1 articles found

 pg 1/13
1 articles found

 pg 2/13
1 articles found

 pg 3/13
1 articles found

 pg 4/13
0 articles found

 pg 5/13
3 articles found

 pg 6/13
1 articles found

 pg 7/13
2 articles found

 pg 8/13
2 articles found

 pg 9/13
0 articles found

 pg 10/13
0 articles found

 pg 11/13
1 articles found

 pg 12/13
0 articles found

 pg 13/13
0 articles found


,Date,Article URL,Headline,Number of Comments,Comments
0,2019-01-17,https://www.nytimes.com/2019/01/16/technology/...,Microsoft Pledges $500 Million for Affordable ...,25,[[Maybe instead of waiting for gestures like t...
1,2019-07-22,https://www.nytimes.com/2019/07/22/technology/...,"With $1 Billion From Microsoft, an A.I. Lab Wa...",25,"[[For the record, artificial Intelligence is t..."
2,2019-11-05,https://www.nytimes.com/2019/11/05/technology/...,Microsoft Surface Pro X Review: Powerful Compu...,25,[[I have been using Surface Pros since the 4th...
3,2020-02-13,https://www.nytimes.com/2020/02/13/technology/...,"Judge Halts Work on Microsoft’s JEDI Contract,...",25,"[[I am not a major fan of Amazon, per se, but ..."
4,2020-07-31,https://www.nytimes.com/2020/07/31/technology/...,"Microsoft Said to Be in Talks to Buy TikTok, a...",25,[[It’s astonishing to read already in the firs...
5,2020-08-03,https://www.nytimes.com/2020/08/03/technology/...,"With TikTok, ‘Uncool’ Microsoft Aims for the L...",25,[[Microsoft is plenty cool enough. Don’t forge...
6,2020-08-03,https://www.nytimes.com/2020/08/03/technology/...,"Trump Reverses Course on TikTok, Opening Door ...",25,[[It's really quite something how this article...
7,2020-08-26,https://www.nytimes.com/2020/08/26/technology/...,How TikTok’s Talks With Microsoft Turned Into ...,10,[[The tactic of mobilizing political support b...
8,2020-09-13,https://www.nytimes.com/2020/09/13/technology/...,"Oracle Chosen as TikTok’s Tech Partner, as Mic...",25,[[This story ignores the elephant in the room....
9,2020-11-05,https://www.nytimes.com/2020/11/05/technology/...,Xbox Series X and S Review: Microsoft’s New Co...,11,[[They are forgetting about the new version of...


In [142]:
getArticlesData('NVIDIA')

Hits: 11
pg 0/1
pg 1/1


,Date,Article URL,Headline,Number of Comments,Comments
0,2021-12-02,https://www.nytimes.com/2021/12/02/technology/...,F.T.C. Sues to Stop Blockbuster Chip Deal Betw...,16,[[This is a very good action. The more that c...


In [197]:
getArticlesData('Tesla')

Hits: 187
pg 0/18
comments – 0
no comments
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
pg 1/18
comments – 0
comments – 25
comments – 0
comments – 0
no comments
no comments
no comments
comments – 0
comments – 0
comments – 0
pg 2/18
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
no comments
comments – 0
pg 3/18
comments – 0
comments – 0
comments – 0
comments – 25
comments – 25
no comments
no comments
comments – 0
comments – 0
no comments
pg 4/18
comments – 0
no comments
comments – 0
comments – 0
comments – 0
comments – 0
comments – 0
no comments
no comments
no comments
pg 5/18
no comments
comments – 0
no comments
comments – 0
comments – 0
comments – 0
comments – 0
no comments
comments – 0
comments – 0
pg 6/18
comments – 0
no comments
comments – 0
no comments
comments – 0
no comments
comments – 0
no comments
comments – 0
comments – 0
pg 7/18
no comments
no comments
comments

,Date,Article URL,Headline,Number of Comments,Comments
0,2019-03-06,https://www.nytimes.com/2019/03/06/business/en...,Tesla’s Troubles Mount: Shuttered Showrooms an...,25,[[Yet another gloom and doom article from the ...
1,2019-06-03,https://www.nytimes.com/2019/06/03/business/de...,Can Tesla Ever Be More Than a Niche Automaker?...,25,[[The Times coverage of Tesla echoes the negat...
2,2019-06-10,https://www.nytimes.com/2019/06/10/business/te...,"Tesla, Facing Setbacks and Skeptics, Tries to ...",25,[[While experts are wondering if Tesla is year...
3,2020-05-19,https://www.nytimes.com/2020/05/19/technology/...,Tesla Owners Try to Make Sense of Elon Musk’s ...,25,"[[I've long been an admirer of Musk, but my ad..."
4,2020-07-22,https://www.nytimes.com/2020/07/22/business/te...,"In Electric Car Market, It’s Tesla and a Jumbl...",25,[[This article completely misses mentioning th...
5,2020-12-22,https://www.nytimes.com/2020/12/22/business/el...,A Race to Become the Tesla of Delivery Trucks ...,25,[[I think it is interesting to note the differ...
6,2021-01-19,https://www.nytimes.com/2021/01/19/business/ri...,The Next Tesla? Investors Bet Big on Electric ...,25,"[[Typewriters are cool, they look good, they s..."
7,2021-03-15,https://www.nytimes.com/2021/03/15/business/vo...,Volkswagen Aims to Use Its Size to Head Off Tesla,25,[[These companies had better all use the same ...
8,2021-04-22,https://www.nytimes.com/2021/04/22/business/el...,Three Electric S.U.V.s With Tesla in Their Sights,25,[[I am surprised that this article does not m...
9,2021-04-29,https://www.nytimes.com/2021/04/29/business/en...,Tesla’s Latest Solar Stumble: Big Price Increases,25,[[I have traditional PV panels covering the so...
